In [1]:
import requests
import urllib.request
import time
import os
import warnings
import re

from bs4 import BeautifulSoup

import csv
import os.path

In [2]:
def generateFileCSV(listHasil,csvName1):

	csvFolrder = "./"
	csvName = csvName1

	if os.path.exists(csvName):

		f = open(csvName, 'a', newline='\n')
		print(f)
		print("ada")
		w = csv.writer(f)

	else:

		f = open(csvName, 'w', newline='\n')
		print(f)
		print("tidak ada")

		w = csv.writer(f)
		w.writerow(("terdakwa", "penuntut_umum", "nomor", "tingkat_proses", "klasifikasi", "kata_kunci", "tahun", "tanggal_register",
              "lembaga_peradilan", "jenis_lembaga_peradilan", "hakim_ketua", "hakim_anggota", "panitera", "amar",
              "amar_lainnya", "catatan_amar", "tanggal_musyawarah", "tanggal_dibacakan", "kaidah", "abstrak", "url"))

	# menulis file csv
	for s in listHasil:
		w.writerow(s)

	f.close()
	berhasil = "\nCreate Csv file Berhasil\n"
	return berhasil

In [3]:
# Fungsi bantuan untuk membersihkan teks dari tag HTML
def clean_html(raw_html):
    # Membersihkan semua tag HTML
    cleanr = re.compile('<.*?>')
    # Membersihkan spasi berlebih dan baris baru
    cleantext = re.sub(cleanr, '', raw_html).strip().replace('\n', ' ').replace('\r', ' ')
    return ' '.join(cleantext.split()) # Menghilangkan spasi ganda

def generateMeta(urlMeta):
    # Memastikan URL bersih dari spasi atau karakter tidak penting
    url = str(urlMeta).strip()
    print(f"Mencoba scraping dari: {url}")
    response = requests.get(url, verify=False, timeout=90) # Tambahkan timeout
    
    # Cek jika halaman berhasil diakses
    if response.status_code != 200:
        print(f"Gagal mengakses URL, status code: {response.status_code}")
        # Kembalikan list kosong sesuai jumlah kolom agar CSV tidak berantakan
        return [""] * 21 

    soup = BeautifulSoup(response.text, 'html.parser')

    # --- Inilah bagian logika scraping yang baru dan sudah diperbaiki ---

    # 1. Buat "kamus" untuk menyimpan hasil. Ini lebih mudah dikelola.
    metadata = {
        "terdakwa": "", "penuntut_umum": "", "nomor": "", "tingkat_proses": "", 
        "klasifikasi": "", "kata_kunci": "", "tahun": "", "tanggal_register": "",
        "lembaga_peradilan": "", "jenis_lembaga_peradilan": "", "hakim_ketua": "", 
        "hakim_anggota": "", "panitera": "", "amar": "", "amar_lainnya": "", 
        "catatan_amar": "", "tanggal_musyawarah": "", "tanggal_dibacakan": "", 
        "kaidah": "", "abstrak": "", "url": url
    }

    # 2. Cari tabel utama yang berisi informasi metadata.
    #    Berdasarkan struktur saat ini, tabel tersebut tidak memiliki class/id yang unik,
    #    tapi berada di dalam <div class="card-body">. Kita akan cari dari sana.
    card_body = soup.find('div', class_='card-body')
    if not card_body:
        print("Peringatan: Elemen <div class='card-body'> tidak ditemukan.")
        # Mengembalikan nilai default jika struktur dasar tidak ada
        return list(metadata.values())

    # Cari semua baris (tr) di dalam tabel di dalam card-body
    rows = card_body.select('table tr')

    # 3. Iterasi (looping) melalui setiap baris untuk mengambil data
    for row in rows:
        # Dapatkan semua sel (th: header, td: data) dalam satu baris
        cells = row.find_all(['th', 'td'])
        if len(cells) >= 2:
            # Ambil teks dari sel header dan sel data, lalu bersihkan
            header = clean_html(str(cells[0])).lower()
            value = clean_html(str(cells[1]))

            # 4. Masukkan data ke dalam kamus berdasarkan headernya
            if 'nomor putusan' in header:
                metadata['nomor'] = value
            elif 'tingkat proses' in header:
                metadata['tingkat_proses'] = value
            elif 'klasifikasi' in header:
                metadata['klasifikasi'] = value
            elif 'kata kunci' in header:
                metadata['kata_kunci'] = value
            elif 'tahun' in header:
                metadata['tahun'] = value
            elif 'tanggal register' in header:
                metadata['tanggal_register'] = value
            elif 'lembaga peradilan' in header:
                metadata['lembaga_peradilan'] = value
            elif 'hakim ketua' in header:
                metadata['hakim_ketua'] = value
            elif 'hakim anggota' in header:
                # Menampung jika ada lebih dari satu hakim anggota
                if metadata['hakim_anggota']:
                    metadata['hakim_anggota'] += "; " + value
                else:
                    metadata['hakim_anggota'] = value
            elif 'panitera' in header:
                metadata['panitera'] = value
            elif 'amar' in header:
                metadata['amar'] = value
            elif 'amar putusan' in header:
                metadata['amar_lainnya'] = value
            elif 'catatan amar' in header:
                metadata['catatan_amar'] = value
            elif 'tanggal musyawarah' in header:
                metadata['tanggal_musyawarah'] = value
            elif 'tanggal dibacakan' in header:
                metadata['tanggal_dibacakan'] = value
            
    # 5. Ekstraksi data Terdakwa/Penuntut (yang strukturnya berbeda)
    #    Data ini seringkali ada di dalam <strong> di atas tabel.
    strong_tags = card_body.find_all('strong')
    for tag in strong_tags:
        text = tag.get_text()
        if 'Terdakwa:' in text:
            # Mengambil teks setelah 'Terdakwa:'
            metadata['terdakwa'] = text.split('Terdakwa:')[1].strip()
        elif 'Penuntut Umum:' in text:
            # Mengambil teks setelah 'Penuntut Umum:'
            metadata['penuntut_umum'] = text.split('Penuntut Umum:')[1].strip()

    # 6. Mengambil Abstrak dan Kaidah Hukum (jika ada)
    #    Biasanya ada di bawah tabel dalam div terpisah
    abstrak_div = soup.find('h4', string=re.compile(r'\s*Abstrak\s*'))
    if abstrak_div and abstrak_div.find_next_sibling('p'):
        metadata['abstrak'] = abstrak_div.find_next_sibling('p').get_text(strip=True)
    
    kaidah_div = soup.find('h4', string=re.compile(r'\s*Kaidah Hukum\s*'))
    if kaidah_div and kaidah_div.find_next_sibling('p'):
        metadata['kaidah'] = kaidah_div.find_next_sibling('p').get_text(strip=True)

    # 7. Mengubah kamus kembali menjadi list sesuai urutan header CSV
    ordered_list_hasil = [
        metadata['terdakwa'], metadata['penuntut_umum'], metadata['nomor'], metadata['tingkat_proses'],
        metadata['klasifikasi'], metadata['kata_kunci'], metadata['tahun'], metadata['tanggal_register'],
        metadata['lembaga_peradilan'], metadata['jenis_lembaga_peradilan'], metadata['hakim_ketua'],
        metadata['hakim_anggota'], metadata['panitera'], metadata['amar'], metadata['amar_lainnya'],
        metadata['catatan_amar'], metadata['tanggal_musyawarah'], metadata['tanggal_dibacakan'],
        metadata['kaidah'], metadata['abstrak'], metadata['url']
    ]

    return ordered_list_hasil

In [4]:
def main():

    warnings.filterwarnings('ignore')

    #folderListURL = pathFile
    fileListURL = "E:\Tugas Kuliah\Semester_lima\Ektraksi Informasi\hasilListURLPage.txt"
    fileMetaCSV = "E:\Tugas Kuliah\Semester_lima\Ektraksi Informasi\hasil.csv"
    listHasil =[]

    startTime = time.time()
    openfileListURL = open(fileListURL, "r", encoding='UTF8')
    bacaListURL = openfileListURL.readlines()

    i = 1
    for barisURL in bacaListURL:
        try:
            #print(str(barisURL))
            hasil = generateMeta(str(barisURL))
            listHasil.append(hasil)
            print("======= ROW HASIL =======",i)
            #print(hasil)
        except Exception as e:
            print(f"Error Get Meta Inf, {e}")
        i=i+1

    createFile = generateFileCSV(listHasil,fileMetaCSV)

    openfileListURL.close()
    endTime = time.time()
    #print(listHasil)
    print('Time Processing : ', endTime-startTime, ' Second')

main();



<>:6: SyntaxWarning: invalid escape sequence '\T'
<>:7: SyntaxWarning: invalid escape sequence '\T'
<>:6: SyntaxWarning: invalid escape sequence '\T'
<>:7: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Syafiq Azizi\AppData\Local\Temp\ipykernel_16552\3906290239.py:6: SyntaxWarning: invalid escape sequence '\T'
  fileListURL = "E:\Tugas Kuliah\Semester_lima\Ektraksi Informasi\hasilListURLPage.txt"
C:\Users\Syafiq Azizi\AppData\Local\Temp\ipykernel_16552\3906290239.py:7: SyntaxWarning: invalid escape sequence '\T'
  fileMetaCSV = "E:\Tugas Kuliah\Semester_lima\Ektraksi Informasi\hasil.csv"


Mencoba scraping dari: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaf088a6cf2c8be88c5d313631373237.html
======= ROW HASIL ======= 1
Mencoba scraping dari: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaf087df132a13748cb6313632373431.html
======= ROW HASIL ======= 2
Mencoba scraping dari: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaf086eac2113aa29f0f313131383438.html
======= ROW HASIL ======= 3
Mencoba scraping dari: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaf082e2c1c4deaca734303831313237.html
======= ROW HASIL ======= 4
Mencoba scraping dari: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaf0775e6b49aab08025313632363235.html
======= ROW HASIL ======= 5
Mencoba scraping dari: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaf073439af6faca8775313130343234.html
======= ROW HASIL ======= 6
Mencoba scraping dari: https://putusan3.mahkamahagung.go.id/direktori/putusan/zaf06c5a071c56bea08e313535373137.html
======= ROW HASIL ==